In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk


class LaneDetector:
    def __init__(self):
        self.previous_left_line = None
        self.previous_right_line = None

    def make_points(self, image, line):
        slope, intercept = line
        height, width, _ = image.shape
        y1 = height  
        y2 = int(y1 * 3 / 5)
        try:
            x1 = int((y1 - intercept) / slope)
            x2 = int((y2 - intercept) / slope)
        except ZeroDivisionError:
            return None
        return [[x1, y1, x2, y2]]

    def average_slope_intercept(self, image, lines):
        left_fit = []
        right_fit = []
        if lines is None:
            return None
        for line in lines:
            for x1, y1, x2, y2 in line:
                fit = np.polyfit((x1, x2), (y1, y2), 1)
                slope, intercept = fit
                if slope < 0:  
                    left_fit.append((slope, intercept))
                else:  # Right line
                    right_fit.append((slope, intercept))

        left_line = right_line = None
        if left_fit:
            left_fit_average = np.average(left_fit, axis=0)
            left_line = self.make_points(image, left_fit_average)
        if right_fit:
            right_fit_average = np.average(right_fit, axis=0)
            right_line = self.make_points(image, right_fit_average)

        return left_line, right_line

    def extrapolate_lines(self, lines, frame_shape):

        height, _, _ = frame_shape
        if lines is None:
            return None

        def extend_line(line):
            if line is None:
                return None
            x1, y1, x2, y2 = line[0]
            slope = (y2 - y1) / (x2 - x1 + 1e-6) 
            intercept = y1 - slope * x1
            y1_new = height
            x1_new = int((y1_new - intercept) / slope)
            y2_new = int(height * 0.55)  
            x2_new = int((y2_new - intercept) / slope)
            return [x1_new, y1_new, x2_new, y2_new]

        return [extend_line(line) for line in lines if line is not None]

    def canny(self, img):
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        median_intensity = np.median(blur)
        lower_threshold = int(max(0, 0.7 * median_intensity))
        upper_threshold = int(min(255, 1.3 * median_intensity))
        return cv2.Canny(blur, lower_threshold, upper_threshold)

    def display_lines(self, img, lines):
        line_image = np.zeros_like(img)
        if lines is not None:
            for line in lines:
                if line is not None:
                    x1, y1, x2, y2 = line
                    cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
        return line_image

    def region_of_interest(self, canny):
        height, width = canny.shape
        mask = np.zeros_like(canny)
        triangle = np.array([[
            (width // 9, height),
            (width * 4 // 9, height // 2),
            (width * 8 // 9, height)
        ]], np.int32)
        cv2.fillPoly(mask, triangle, 255)
        return cv2.bitwise_and(canny, mask)

    def process_frame(self, frame):
        canny_image = self.canny(frame)
        cropped_canny = self.region_of_interest(canny_image)
        lines = cv2.HoughLinesP(cropped_canny, 2, np.pi / 180, 100, np.array([]), minLineLength=30, maxLineGap=225)
        averaged_lines = self.average_slope_intercept(frame, lines)

        # Use previous lines if current detection fails
        if averaged_lines is None:
            left_line, right_line = self.previous_left_line, self.previous_right_line
        else:
            left_line, right_line = averaged_lines
            self.previous_left_line, self.previous_right_line = left_line, right_line

        # Extrapolate incomplete lines
        extrapolated_lines = self.extrapolate_lines([left_line, right_line], frame.shape)

        line_image = self.display_lines(frame, extrapolated_lines)
        return cv2.addWeighted(frame, 0.8, line_image, 1, 1)

# GUI Implementation
class LaneDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Lane Detection")
        self.detector = LaneDetector()


        self.cap = cv2.VideoCapture("test2.mp4")
        self.running = False

    
        _, frame = self.cap.read()
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        self.frame_height, self.frame_width, _ = frame.shape

     
        self.resize_factor = 0.5
        self.resized_width = int(self.frame_width * self.resize_factor)
        self.resized_height = int(self.frame_height * self.resize_factor)

     
        self.original_canvas = tk.Canvas(root, width=self.resized_width, height=self.resized_height)
        self.original_canvas.grid(row=0, column=0, padx=10, pady=10)

        self.processed_canvas = tk.Canvas(root, width=self.resized_width, height=self.resized_height)
        self.processed_canvas.grid(row=0, column=1, padx=10, pady=10)

        # Control buttons
        self.control_frame = ttk.Frame(root)
        self.control_frame.grid(row=1, column=0, columnspan=2)

        self.start_button = ttk.Button(self.control_frame, text="Start", command=self.start_video)
        self.start_button.grid(row=0, column=0, padx=10)

        self.stop_button = ttk.Button(self.control_frame, text="Stop", command=self.stop_video)
        self.stop_button.grid(row=0, column=1, padx=10)

        self.exit_button = ttk.Button(self.control_frame, text="Exit", command=root.quit)
        self.exit_button.grid(row=0, column=2, padx=10)

        self.root.geometry(f"{2 * self.resized_width + 40}x{self.resized_height + 100}")

    def start_video(self):
        self.running = True
        self.update_frame()

    def stop_video(self):
        self.running = False

    def update_frame(self):
        if not self.running:
            return

        ret, frame = self.cap.read()
        if not ret:
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            return

        resized_frame = cv2.resize(frame, (self.resized_width, self.resized_height))
        processed_frame = self.detector.process_frame(frame)
        resized_processed_frame = cv2.resize(processed_frame, (self.resized_width, self.resized_height))

        original_image = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
        processed_image = cv2.cvtColor(resized_processed_frame, cv2.COLOR_BGR2RGB)

        original_image = ImageTk.PhotoImage(Image.fromarray(original_image))
        processed_image = ImageTk.PhotoImage(Image.fromarray(processed_image))

        self.original_canvas.create_image(0, 0, image=original_image, anchor=tk.NW)
        self.processed_canvas.create_image(0, 0, image=processed_image, anchor=tk.NW)

        self.original_canvas.image = original_image
        self.processed_canvas.image = processed_image

        self.root.after(10, self.update_frame)


if __name__ == "__main__":
    root = tk.Tk()
    app = LaneDetectionApp(root)
    root.mainloop()



2024-12-01 23:22:35.998 python[26787:2537886] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-01 23:22:35.998 python[26787:2537886] +[IMKInputSession subclass]: chose IMKInputSession_Modern
